# Frequency response

Import modules and configure the notebook.

In [1]:
import os

# This module is part of the python standard library
import time

# These modules are part of other existing libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

from scipy import linalg
from scipy import signal

# This is my own script (it is an interface to the pybullet simulator)
import ae353_drone

# I often go back and forth between making changes to my scripts and to
# the notebook in which they are used. One "gotcha" is that notebooks only
# import modules or scripts ONCE. Subsequent imports don't do anything, and
# in particular won't reflect any changes I've made to my scripts. To make
# sure that I'm working with the latest version of my code, I use this bit
# of magic, which forces the notebook to "reload" my script:
import importlib
importlib.reload(ae353_drone)

<module 'ae353_drone' from '/Users/timothybretl/Documents/courses/AE353/09 - AE353 (Spring 2021)/Website/examples/day41_drone/ae353_drone.py'>

Create simulator.

In [2]:
simulator = ae353_drone.Simulator(display=False, pos_noise=0., rpy_noise=0.)

Design controller and observer.

In [3]:
def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @  B.T @ P
    return K

A = np.array([[0., 1.], [0., 0.]])
B = np.array([[0.], [2.]])
C = np.array([[1., 0.]])

# Choose gains
Qc = np.diag([100., 1.])
Rc = np.diag([1.])

# Find optimal gain matrix
K = lqr(A, B, Qc, Rc)
print(f'Gain matrix of controller:\n K = np.array({K.tolist()})\n')

# Choose gains
Qo = np.diag([1.])
Ro = np.diag([1., 1.])

# Find optimal gain matrix
L = lqr(A.T, C.T, linalg.inv(Ro), linalg.inv(Qo)).T
print(f'Gain matrix of observer:\n L = np.array({L.tolist()})\n')

Gain matrix of controller:
 K = np.array([[9.999999999999996, 3.316624790355398]])

Gain matrix of observer:
 L = np.array([[1.732050807568877], [0.9999999999999993]])



Find closed-loop system:

$$
\begin{align*}
\begin{bmatrix} \dot{x} \\ \dot{x}_\text{err} \end{bmatrix}
&=
\begin{bmatrix} A - BK & -BK \\ 0 & A - LC \end{bmatrix}
\begin{bmatrix} x \\ x_\text{err} \end{bmatrix}
+
\begin{bmatrix} BKM \\ 0 \end{bmatrix} r \\
y
&=
\begin{bmatrix} C & 0 \end{bmatrix}
\begin{bmatrix} x \\ x_\text{err} \end{bmatrix}
+
\begin{bmatrix} 0 \end{bmatrix} r
\end{align*}
$$

where

$$
M = \begin{bmatrix} 1 \\ 0 \end{bmatrix}
$$

and where $r$ is the desired height.

In [4]:
kRef = K @ np.array([[1.], [0.]])

Am = np.block([[A - B @ K, - B @ K], [np.zeros((2, 2)), A - L @ C]])
Bm = np.block([[B @ kRef], [np.zeros((2, 1))]])
Cm = np.block([[C, np.zeros((1, 2))]])

sysm = signal.StateSpace(Am, Bm, Cm, 0.)

Find frequency response. (Ignore the warning.)

In [5]:
w, H = signal.freqresp(sysm)
mag = np.absolute(H)
phase = np.angle(H)

/Users/timothybretl/Applications/miniconda3/envs/ae353-bullet/lib/python3.9/site-packages/scipy/signal/filter_design.py:1625: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  warnings.warn("Badly conditioned filter coefficients (numerator): the "


Plot frequency response (i.e., make a Bode plot).

In [6]:
fig, (ax_mag, ax_phase) = plt.subplots(2, 1, figsize=(9, 9), sharex=True)
ax_mag.loglog(w, mag)
ax_mag.grid()
ax_mag.tick_params(labelsize=14)
ax_mag.set_ylabel('magnitude', fontsize=16)
ax_phase.semilogx(w, phase)
ax_phase.grid()
ax_phase.tick_params(labelsize=14)
ax_phase.set_ylabel('angle (radians)', fontsize=16)
ax_phase.set_ylim([-np.pi, 0.])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-3.141592653589793, 0.0)

Get magnitude and phase at one particular frequency. (Ignore the warning.)

In [7]:
w = (2 * np.pi / 1.)
print(f'w = {w:.3f}')
w, H = signal.freqresp(sysm, w=w)
mag = np.absolute(H)
phase = np.angle(H)

print(f'mag = {mag.item():.3f}\nphase = {phase.item():.3f}')

w = 6.283
mag = 0.435
phase = -2.008


/Users/timothybretl/Applications/miniconda3/envs/ae353-bullet/lib/python3.9/site-packages/scipy/signal/filter_design.py:1625: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  warnings.warn("Badly conditioned filter coefficients (numerator): the "


Implement controller.

In [15]:
class RobotController:
    def __init__(self, limiter=None):
        self.dt = 0.01
        self.limiter = limiter
        
        # Initialize record of old measurements
        self.r_old = 0.
        self.p_old = 0.
        self.y_old = 0.
        
        self.A = np.array([[0., 1.], [0., 0.]])
        self.B = np.array([[0.], [2.]])
        self.C = np.array([[1., 0.]])
        self.K = np.array([[9.999999999999996, 3.316624790355398]])
        self.L = np.array([[1.732050807568877], [0.9999999999999993]])
        
        self.f_z_e = 0.5 * 9.81
        self.z_e = 0.
        

    def get_color(self):
        return [0., 1., 0.]

    def reset(self, pos):
        self.xhat = np.zeros(2)
        self.t = 0.
        self.user_data = {
            'tau_x_des': 0.,
            'tau_y_des': 0.,
            'tau_z_des': 0.,
            'f_z_des': 0.,
            'zdes': 0.,
        }

    def run(self, pos, rpy, pos_ring, is_last_ring, pos_others):
        
        ##################
        # PD Control
        #
        #  This is easy to implement and allows us to isolate the
        #  "z position and velocity" subsystem, for the purpose of
        #  example.
        #
        
        # Get current measurements of roll, pitch, and yaw
        r = rpy[0]
        p = rpy[1]
        y = rpy[2]
        
        # Estimate roll, pitch, and yaw derivatives by finite difference
        rdot = (r - self.r_old) / self.dt
        pdot = (p - self.p_old) / self.dt
        ydot = (y - self.y_old) / self.dt
        
        # Update record of old measurements
        self.r_old = r
        self.p_old = p
        self.y_old = y
        
        # Choose net torques to drive roll, pitch, and yaw to zero
        tau_x = - 1. * (r - 0) - 0.1 * (rdot - 0)
        tau_y = - 1. * (p - 0) - 0.1 * (pdot - 0)
        tau_z = - 1. * (y - 0) - 0.1 * (ydot - 0)
        
        #
        ##################
        
        
        w = (2 * np.pi) / 2.
        zdes = 1. + 0.1 * np.sin(w * self.t)
        self.user_data['zdes'] = zdes
        self.t += self.dt
        
#         zdes = pos_ring[2]
        zest = self.xhat[0] + self.z_e
        max_error = 0.25
        if np.abs(zdes - zest) > max_error:
            zdes = zest + max_error * ((zdes - zest) / linalg.norm(zdes - zest))
        
        
        xdes = np.array([zdes - self.z_e, 0.])
        u = -self.K @ (self.xhat - xdes)
        f_z = u[0] + self.f_z_e
        
        self.user_data['tau_x_des'] = tau_x
        self.user_data['tau_y_des'] = tau_y
        self.user_data['tau_z_des'] = tau_z
        self.user_data['f_z_des'] = f_z
        
        if self.limiter is not None:
            tau_x, tau_y, tau_z, f_z = self.limiter(tau_x, tau_y, tau_z, f_z)
        u[0] = f_z - self.f_z_e
        
        y = np.array([pos[2] - self.z_e])
        self.xhat += self.dt * (self.A @ self.xhat + self.B @ u - self.L @ (self.C @ self.xhat - y))

        return tau_x, tau_y, tau_z, f_z

Run simulation.

In [16]:
simulator.clear_drones()
simulator.add_drone(RobotController, 'my_netid', 'my_image.png')
simulator.reset()
simulator.run(max_time=10.0)

Get drone by name.

In [17]:
drone_name = 'my_netid'
drone = simulator.get_drone_by_name(drone_name)

if drone is None:
    drone_names = '\n'.join([d['name'] for d in simulator.drones])
    msg = f'The simulator has no drone with name "{drone_name}".'
    if len(drone_names) == 0:
        msg += f' The simulator has no drones at all, in fact.'
    else:
        msg += f' The simulator has these drones:'
        msg += f'\n==========\n{drone_names}\n==========\n'
    print(msg)

Extract data.

In [18]:
data = drone['data'].copy()

Convert all lists in data to numpy arrays.

In [19]:
for key in data.keys():
    if key != 'user_data':
        data[key] = np.array(data[key]).T
for key in data['user_data'].keys():
    data['user_data'][key] = np.array(data['user_data'][key]).T

In [20]:
data['user_data']

{'tau_x_des': array([ 1.46918483e-03,  8.65834299e-04,  4.23504809e-04,  1.57775168e-04,
         2.85805406e-05, -2.37976405e-05, -4.03703471e-05, -4.25375394e-05,
        -3.98077978e-05, -3.58787627e-05, -3.19845905e-05, -2.84429723e-05,
        -2.52819175e-05, -2.24615049e-05, -1.99387572e-05, -1.76799143e-05,
        -1.56587872e-05, -1.38535285e-05, -1.22445557e-05, -1.08136119e-05,
        -9.54348026e-06, -8.41797908e-06, -7.42203386e-06, -6.54173907e-06,
        -5.76438234e-06, -5.07843039e-06, -4.47348499e-06, -3.94021881e-06,
        -3.47029932e-06, -3.05630724e-06, -2.69165389e-06, -2.37050060e-06,
        -2.08768208e-06, -1.83863483e-06, -1.61933128e-06, -1.42621967e-06,
        -1.25616980e-06, -1.10642426e-06, -9.74554795e-07, -8.58423532e-07,
        -7.56148519e-07, -6.66073233e-07, -5.86739628e-07, -5.16864356e-07,
        -4.55317804e-07, -4.01105635e-07, -3.53352538e-07, -3.11287929e-07,
        -2.74233377e-07, -2.41591545e-07, -2.12836454e-07, -1.87504937e-07,

Plot difference between height and desired height.

In [21]:
# Create a figure with subplots that all share the same x-axis
fig, (ax_z) = plt.subplots(1, 1, figsize=(9, 6), sharex=True)

z_tru = data['pos'][2, :]
z_des = data['user_data']['zdes']

# Position
ax_z.plot(data['t'], z_tru, label='p_z (true)', linewidth=4)
ax_z.plot(data['t'], z_des, '--', label='p_z (desired)', linewidth=4)
ax_z.grid()
ax_z.legend(fontsize=16)
ax_z.tick_params(labelsize=14)
ax_z.set_ylim([0.8, 1.2])

# Set shared x-axis properties
ax_z.set_xlabel('time (s)', fontsize=20)
ax_z.set_xlim([data['t'][0], data['t'][-1]])

# Make the arrangement of subplots look nice
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
- ((41.563 - 41.249) / (41.563 - 40.563)) * 2 * np.pi

In [ ]:
- ((42.857 - 42.5) / (42.857 - 32.857)) * 2 * np.pi

Plot difference between state and state estimate.

In [ ]:
# Create a figure with subplots that all share the same x-axis
fig, (ax_z, ax_v) = plt.subplots(2, 1, figsize=(8, 6), sharex=True)

z_tru = data['pos'][2, :]
z_est = data['xhat'][0, :]
z_des = data['user_data']['zdes']

v_tru = data['linvel'][2, :]
v_est = data['xhat'][1, :]


# Position
ax_z.plot(data['t'], z_tru, label='p_z (true)', linewidth=4)
ax_z.plot(data['t'], z_est, '--', label='p_z (estimate)', linewidth=4)
ax_z.plot(data['t'], z_des, ':', label='p_z (desired)', linewidth=4)
ax_z.grid()
ax_z.legend(fontsize=16)
ax_z.tick_params(labelsize=14)
ax_z.set_ylim([0., 2.])

# Velocity
ax_v.plot(data['t'], v_tru, label='v_z (true)', linewidth=4)
ax_v.plot(data['t'], v_est, '--', label='v_z (estimate)', linewidth=4)
ax_v.grid()
ax_v.legend(fontsize=16)
ax_v.tick_params(labelsize=14)
ax_v.set_ylim([-1., 1.])

# Set shared x-axis properties
ax_v.set_xlabel('time (s)', fontsize=20)
ax_v.set_xlim([data['t'][0], data['t'][-1]])

# Make the arrangement of subplots look nice
fig.tight_layout()

Plot other results.

In [ ]:
# Create a figure with subplots that all share the same x-axis
fig, (ax_pos, ax_rpy, ax_act) = plt.subplots(3, 1, figsize=(9, 12), sharex=True)

# Position
ax_pos.plot(data['t'], data['pos'][0, :], label='x (m)', linewidth=4)
ax_pos.plot(data['t'], data['pos'][1, :], label='y (m)', linewidth=4)
ax_pos.plot(data['t'], data['pos'][2, :], label='z (m)', linewidth=4)
ax_pos.grid()
ax_pos.legend(fontsize=16)
ax_pos.tick_params(labelsize=14)

# Roll, pitch, and yaw angles
ax_rpy.plot(data['t'], data['rpy'][0, :], label='roll (rad)', linewidth=4)
ax_rpy.plot(data['t'], data['rpy'][1, :], label='pitch (rad)', linewidth=4)
ax_rpy.plot(data['t'], data['rpy'][2, :], label='yaw (rad)', linewidth=4)
ax_rpy.grid()
ax_rpy.legend(fontsize=16)
ax_rpy.tick_params(labelsize=14)

# Actuator commands
ax_act.plot(data['t'], data['tau_x'], label='tau_x (N-m)', linewidth=4)
ax_act.plot(data['t'], data['user_data']['tau_x_des'], '--', label='desired tau_x (N-m)', linewidth=4)
ax_act.plot(data['t'], data['tau_y'], label='tau_y (N-m)', linewidth=4)
ax_act.plot(data['t'], data['user_data']['tau_y_des'], '--', label='desired tau_y (N-m)', linewidth=4)
ax_act.plot(data['t'], data['tau_z'], label='tau_z (N-m)', linewidth=4)
ax_act.plot(data['t'], data['user_data']['tau_z_des'], '--', label='desired tau_z (N-m)', linewidth=4)
ax_act.plot(data['t'], data['f_z'], label='f_z (N)', linewidth=4)
ax_act.plot(data['t'], data['user_data']['f_z_des'], '--', label='desired f_z (N-m)', linewidth=4)
ax_act.grid()
ax_act.legend(fontsize=16)
ax_act.tick_params(labelsize=14)

# Set shared x-axis properties
ax_act.set_xlabel('time (s)', fontsize=20)
ax_act.set_xlim([data['t'][0], data['t'][-1]])

# Make the arrangement of subplots look nice
fig.tight_layout()